In [1]:
#Import the relevand section of the NLP library
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize

import re

from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/Alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
seinfeld_directory = 'Seinfeld_Episodes/Season_5/'

seinfeld_season_5_episodes = ['S05_E01_The_Mango.txt', 'S05_E02_The_Puffy_Shirt.txt',
                              'S05_E03_The_Glasses.txt', 'S05_E04_The_Sniffing_Accountant.txt',
                              'S05_E05_The_Bris.txt', 'S05_E06_The_Lip_Reader.txt',
                              'S05_E07_The_Non_Fat_Yogurt.txt', 'S05_E08_The_Barber.txt',
                              'S05_E09_The_Masseuse.txt', 'S05_E10_The_Cigar_Store_Indian.txt',
                              'S05_E11_The_Conversion.txt', 'S05_E12_The_Stall.txt',
                              'S05_E13_The_Dinner_Party.txt', 'S05_E14_The_Marine_Biologist.txt',
                              'S05_E15_The_Pie.txt', 'S05_E16_The_Stand-In.txt',
                              'S05_E17_The_Wife.txt', 'S05_E18_The_Raincoats_Part_1.txt',
                              'S05_E19_The_Raincoats_Part_2.txt', 'S05_E20_The_Fire.txt',
                              'S05_E21_The_Hamptons.txt', 'S05_E22_The Opposite.txt']

In [3]:
def open_file(file_name):

    with open(seinfeld_directory + file_name, 'r') as file:
        raw_text = file.read().replace('\n', ' ')
    
    return raw_text

In [4]:
raw_text_episode_2 = open_file(seinfeld_season_5_episodes[1])

In [ ]:
# def cleaned_episode(raw_text):
    
#     raw_text_no_notes = re.sub("[\(\[].*?[\)\]]", "", raw_text)

#     for symbol in ",.?!''\n":
#         cleaned_text = raw_text_no_notes.replace(symbol, "").lower()

#     cleaned_text = cleaned_text.split(" ")    
    
#     for i in cleaned_text:
        
#         if i.endswith(':') == True or i == '' or i == ' ':
#             cleaned_text.remove(i)   
     
#     return cleaned_text

In [16]:
def cleaned_episode(raw_text):
    
    raw_text_no_notes = re.sub("[\(\[].*?[\)\]]", "", raw_text)

    for symbol in "*,.?!''\n":
        raw_text_no_notes = raw_text_no_notes.replace(symbol, '').lower()
  
    cleaned_text = raw_text_no_notes.split(" ")    
    
    for i in cleaned_text:
        
        if i.endswith(':') == True or i == '' or i == ' ':
            cleaned_text.remove(i)
            
        i = i.replace('.', '')
        i = i.replace('?', '')
        i = i.replace('!', '')

     
    return cleaned_text

In [17]:
for i in seinfeld_season_5_episodes:
    episode_text = open_file(i)
    print(cleaned_episode(episode_text)[:20])

['a', 'female', 'orgasm', 'is', 'kinda', 'like', 'the', 'bat', 'cave', 'a', 'very', 'few', 'people', 'know', 'where', 'it', 'is', 'and', 'if', 'youre']
['i', 'cant', 'believe', 'this', 'oh', 'it', 'wont', 'be', 'for', 'that', 'long', 'how', 'can', 'i', 'do', 'this', 'how', 'can', 'i', 'move']
['i', 'never', 'get', 'enough', 'sleep', 'i', 'stay', 'up', 'late', 'at', 'night', 'cause', 'im', 'night', 'guy', 'night', 'guy', 'wants', 'to', 'stay']
['jerry', 'george', 'and', 'elaine', 'at', 'monks', 'caf', 'so', 'does', 'he', 'like', 'you', 'what', 'do', 'you', 'think', 'you', 'like', 'him', 'yeah']
['act', 'one', 'scene', 'a', 'int', 'hospital', 'room', '-', 'day', 'jerry', 'elaine', 'and', 'george', 'are', 'visiting', 'with', 'new', 'parents', 'myra', 'and']
['at', 'the', 'comedy', 'club', 'professional', 'tennis', 'to', 'me', 'i', 'dont', 'understand', 'all', 'the', 'shushing', 'why', 'are', 'they', 'always', 'shushing', 'shh']
['jerry', 'ive', 'always', 'been', 'a', 'big', 'fan', 'of', '

# Part Two: tokenize/stemming/lemmatization

## Tokenize

In [27]:
episode_1_text = open_file(seinfeld_season_5_episodes[0])
episode_1_text[:500]

"[location: nightclub] JERRY: A female orgasm is kinda like the bat cave. A very few people know where it is and if you're lucky enough to see it you probably don't know how you got there and you can't find you way back after you left. You know there are two types of female orgasm: the real and the fake. And I'll tell you right now, as a man, we don't know. We do not know, because to man sex is like a car accident and determining the female orgasm is like being asked 'What did you see after the c"

In [33]:
cleaned_episode_1_text = cleaned_episode(episode_1_text)
" ".join(cleaned_episode_1_text[:500])

'a female orgasm is kinda like the bat cave a very few people know where it is and if youre lucky enough to see it you probably dont know how you got there and you cant find you way back after you left you know there are two types of female the real and the fake and ill tell you right now as a man we dont know we do not know because to man sex is like a car accident and determining the female orgasm is like being asked what did you see after the car went out of control i heard a lot of screeching sounds i remember i was facing the wrong way at one point and in the end my body was thrown clear so whats her name karin is she nice great so you like her i think so you dont know i cant tell anymore well do you feel anything feel whats that all right let me ask you when she comes over youre cleaning up a lot yeah youre just straightening up or youre cleaning cleaning you do the tub yeah on your knees ajax scrubbing the whole deal yeah okay i think youre in love tub is love tub is love so the

In [34]:
def tokenize(clean_episode_text):
    joined_episode = ' '.join(clean_episode_text)
    tokenized_episode = word_tokenize(joined_episode)
    
    return tokenized_episode

In [37]:
tokenize_episode_1_text = tokenize(cleaned_episode_1_text)
" ".join(tokenize_episode_1_text[:500])

'a female orgasm is kinda like the bat cave a very few people know where it is and if youre lucky enough to see it you probably dont know how you got there and you cant find you way back after you left you know there are two types of female the real and the fake and ill tell you right now as a man we dont know we do not know because to man sex is like a car accident and determining the female orgasm is like being asked what did you see after the car went out of control i heard a lot of screeching sounds i remember i was facing the wrong way at one point and in the end my body was thrown clear so whats her name karin is she nice great so you like her i think so you dont know i cant tell anymore well do you feel anything feel whats that all right let me ask you when she comes over youre cleaning up a lot yeah youre just straightening up or youre cleaning cleaning you do the tub yeah on your knees ajax scrubbing the whole deal yeah okay i think youre in love tub is love tub is love so the

## Lemmetization

In [54]:
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

import nltk
from nltk.stem import WordNetLemmatizer 

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

In [55]:
tokenize_episode_1_text

['a',
 'female',
 'orgasm',
 'is',
 'kinda',
 'like',
 'the',
 'bat',
 'cave',
 'a',
 'very',
 'few',
 'people',
 'know',
 'where',
 'it',
 'is',
 'and',
 'if',
 'youre',
 'lucky',
 'enough',
 'to',
 'see',
 'it',
 'you',
 'probably',
 'dont',
 'know',
 'how',
 'you',
 'got',
 'there',
 'and',
 'you',
 'cant',
 'find',
 'you',
 'way',
 'back',
 'after',
 'you',
 'left',
 'you',
 'know',
 'there',
 'are',
 'two',
 'types',
 'of',
 'female',
 'the',
 'real',
 'and',
 'the',
 'fake',
 'and',
 'ill',
 'tell',
 'you',
 'right',
 'now',
 'as',
 'a',
 'man',
 'we',
 'dont',
 'know',
 'we',
 'do',
 'not',
 'know',
 'because',
 'to',
 'man',
 'sex',
 'is',
 'like',
 'a',
 'car',
 'accident',
 'and',
 'determining',
 'the',
 'female',
 'orgasm',
 'is',
 'like',
 'being',
 'asked',
 'what',
 'did',
 'you',
 'see',
 'after',
 'the',
 'car',
 'went',
 'out',
 'of',
 'control',
 'i',
 'heard',
 'a',
 'lot',
 'of',
 'screeching',
 'sounds',
 'i',
 'remember',
 'i',
 'was',
 'facing',
 'the',
 'wrong'

In [56]:
lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in tokenize_episode_1_text])
print(lemmatized_output)

a female orgasm is kinda like the bat cave a very few people know where it is and if youre lucky enough to see it you probably dont know how you got there and you cant find you way back after you left you know there are two type of female the real and the fake and ill tell you right now a a man we dont know we do not know because to man sex is like a car accident and determining the female orgasm is like being asked what did you see after the car went out of control i heard a lot of screeching sound i remember i wa facing the wrong way at one point and in the end my body wa thrown clear so whats her name karin is she nice great so you like her i think so you dont know i cant tell anymore well do you feel anything feel whats that all right let me ask you when she come over youre cleaning up a lot yeah youre just straightening up or youre cleaning cleaning you do the tub yeah on your knee ajax scrubbing the whole deal yeah okay i think youre in love tub is love tub is love so there you a